# Precision, Recall und Accuracy

In diesem Teil beschäftigst du dich damit, wie du die Abstraktionsfähigkeit eines Modells und damit seine Vorhersagequalität mit Gütekriterien messen kannst.

Du kennst schon die Confusion Matrix, damit verstehst du im Detail, wie ein Klassifikator sich verhält. Oft möchtest du das aber auf eine (oder mehrere) Kenngröße/n reduzieren, die du in diesem Teil kennenlernen wirst.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In diesem Teil betrachtest du nur die Klassifikation nach Autoren. Genauso kannst du das natürlich für Keywords (einzeln!) oder für die Kommentare durchführen:

## Daten für Autoren-Klassifikation vorbereiten

Du bestimmst zunächst die Top-Autoren:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

Alle Autoren sollen gleich viele Artikel erhalten:

In [ ]:
min_articles = min(top_authors["title"])
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

Nun kannst du die Daten vektorisieren:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

Auch zur Berechnung der Qualitätsmetriken benötigst du einen Split in Trainings- und Testdaten

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, adf["author"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=adf["author"].values)

## Training und Auswertung SVM

Das Training kennst du schon:

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svc = svm.fit(X_train, y_train)

Lass dir nochmal ausrechnen, wie oft der Klassifikator richtig oder falsch liegt:

In [ ]:
import numpy as np
pred_test  = svc.predict(X_test)
np.unique(pred_test == y_test, return_counts=True)

## Metriken

Häufig wird in Klassifikationsaufgaben (oder allemeiner beim Information Retrieval) die sog. Accuracy als Kenngröße angegeben. Dabei handelt es sich um den Anteil der richtig klassifizierten Ergebnisse am Gesamtanteil:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_test)

Neben der Accuracy gibt es aber noch weitere Metriken, die besonders in *schiefen* Datensets eine Rolle spielen. Als Beispiel betrachten wir einen Test für eine sehr seltene Krankheit. Wenn du diesen so "implementierst", dass er immer *negativ* zurückgibt, würdest du eine sehr hohe Accuracy erhalten. Dabei funktioniert der Test gar nicht.

Dafür gibt es andere Maßzahlen, nämlich [Precision und Recall](https://de.wikipedia.org/wiki/Beurteilung_eines_bin%C3%A4ren_Klassifikators#Anwendung_im_Information_Retrieval). Precision (oder auch Spezifizität) gibt dabei an, wie viele der gefundenen Ergebnisse richtig klassifiziert wurden. Recall sagt dir, wie viele der gewünschten Ergebnisse gefunden werden konnten. Leider arbeiten die beiden etwas gegeneinander.

Betrachte dazu das Beispiel und lass dir einen Report ausgeben:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_test))

Wie du siehst, wurden z.B. die Artikel von `Alexander Neumann` fast alle richtig klassifiziert (`94%`), allerdings wurden nur `50%` gefunden. Schlechter sieht es z.B. bei `Andreas Wilkens` aus und ganz schlecht (du ahnst es) bei `Tilman Wittenhorst`. Von ihm wurden zwar immerhin `41%` der Artikel gefunden, aber nur `11%` der ihm zugeordneten Artikel waren überhaut von ihm (das hast du in der hellen Spalte der Confusion Matrix gesehen).

Bei anderen Autoren wie `Isabel Grünwald` und `Mark Mantel` hat das sehr viel besser funktioniert. Vermutlich liegt das daran, dass sie  eine deutlichere Themenausrichtung haben und daher einige Keywords erkannt wurden.

## Vergleich der Klassifikatoren

Mit diesem Handwerkszeug ausgestattet kannst du nun überprüfen, wie sich die anderen Klassifikatoren verhalten:

In [ ]:
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

for clf_class in [SVC, SGDClassifier, MultinomialNB, DecisionTreeClassifier,
            RandomForestClassifier, GradientBoostingClassifier]:
    clf = clf_class(random_state=42) if clf_class != MultinomialNB else clf_class()
    clf.fit(X_train, y_train)
    pred_test  = clf.predict(X_test)
    print(clf_class.__name__)
    print(accuracy_score(y_test, pred_test), 
          precision_score(y_test, pred_test, average='weighted'), 
          recall_score(y_test, pred_test, average='weighted'))
    print("\n")

Die identischen Werte für `accuracy` und den gewichteten `recall` sind auf den ersten Blick erstaunlich - das liegt aber an der Konstruktion unseres "ausgeglichenen" Trainingssets.

Der `SGDClassifier` funktioniert offenbar am besten. Du kannst dir nochmal genau ausgeben lassen, wie die Performance zu bewerten ist:

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
pred_test  = sgd.predict(X_test)
print(classification_report(y_test, pred_test))

Wie du siehst, ist der `SGDClassifier` besonders im `recall` deutlich besser als `SVC`.  Abhängig von deinem Anwendungsfall kann es besser sein, entweder Precision oder Recall zu optimieren.

Wenn du möchtest, versuche auch den `TfidfVectorizer` zu optimieren, indem du z.B. noch Bigramme zulässt.

## Gütekriterien sind unbedingt notwendig

Ein Klassifikationsmodell (oder auch ein Regressionsmodell) ohne Gütekriterien ist nur wenig wert. Leider wirst du in der Praxis häufig sehen, dass das einfach vernachlässigt wird. Solchen Modellen solltest du wenig Glauben schenken.

Du solltest immer darauf achten, in deiner eigenen Arbeit mit diesen Qualitätskriterien zu arbeiten und sie zu berechnen. Es ist besser, du weißt, dass ein Modell *nicht funktioniert* als deine Kollegen damit arbeiten zu lassen.